In [ ]:
import numpy as np
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision as tv

In [ ]:
"""
Create dummy dataset
"""
BATCH_SIZE = 512
NUM_DATA = BATCH_SIZE * 10
NUM_CLASSES = 10
DIM = 100
LR = 1e-2

DEVICE = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"

dummy_data = np.random.normal(size=(NUM_DATA, DIM)).astype(np.float32)
# (NUM_CLASS - 1e-10) is to avoid the random variable is 10.
dummy_label = np.random.uniform(0, NUM_CLASSES - 1e-10, size=(NUM_DATA,)).astype(int) # 

print(dummy_data.shape, dummy_label.shape)

print(DEVICE)


(5120, 100) (5120,)
cuda:0


In [ ]:
"""
Dataset implementation
"""
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, transformations):
        # data is composed of [xs, ys]
        # where xs.shape = [size of total data, dimension]
        # where ys.shape = [size of total data]

        # transformations: [transformation] data augmentation for data
        # transformations[0](data)
        self.data = data
        self.transformations = transformations

        assert len(self.data) == len(self.transformations)

    def __getitem__(self, index):
        return tuple(transformation(data_[index]) if transformation else data_[index] 
                     for data_, transformation in zip(self.data, self.transformations))

    def __len__(self):
        return self.data[0].shape[0]

In [ ]:
"""
Dataset check
"""
def check_dataset():
    dset = MyDataset([dummy_data, dummy_label], [None, None])
    # transformations[0] for dummy_data
    # transformations[1] for dummy_label
    num_data = 0

    for data_ in dset:
        
        input_, label_ = data_
        assert input_.shape == (DIM, )

        num_data += 1

    assert num_data == dummy_data.shape[0], "Implementation of __len__ might be wrong"

    print("pass DataSet check")

check_dataset()

pass DataSet check


In [ ]:
"""
DataLoader implementation
"""
def getData_dataloader(x, y, batchSize):
    # Return a dataloader object
    x, y = torch.FloatTensor(x), torch.LongTensor(y)
    
    x_transformation = None

    y_transformation = None

    d = MyDataset([x, y], [x_transformation, y_transformation])

    ### TODO 
    # Create a dataloader object and return it

    return torch.utils.data.DataLoader(d, batch_size=batchSize, shuffle=True, num_workers=2)


In [ ]:
"""
DataLoader check
"""

def check_dataloader():
    dloader = getData_dataloader(dummy_data, dummy_label, 32)
    num_data = 0

    try:
        for data_ in dloader:
            
            input_, label_ = data_

            num_data += input_.shape[0]

        assert num_data == dummy_data.shape[0], "Implementation of __len__ might be wrong"

        print("pass DataLoader check")

    except IndexError:
        # out of index in data loader
        print("Implementation of __len__ might be wrong")


check_dataloader()

pass DataLoader check


In [ ]:
"""
Model
"""

class BaseModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(BaseModel, self).__init__()
        self.input_size = input_size

        dim = 32
        self.model = nn.Sequential(
            nn.Linear(input_size, dim),
            nn.ReLU(),
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, num_classes),
        )

    def forward(self, x):
        return self.model(x)

model = BaseModel(DIM, NUM_CLASSES).to(DEVICE)

print(model)
print(DEVICE)

BaseModel(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=10, bias=True)
  )
)
cuda:0


In [ ]:
"""
Create the optimizer and objective function
"""
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

"""
Evaluation metrics
"""
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

def genMetrics(trueY, predY):
    """
    Return acc and auc
    """
    accuracy = accuracy_score(trueY, predY)
    
    return round(accuracy, 4)

In [ ]:
"""
Data Module
""" 

# Naive method
def getData_naive(x, y, batchSize):
    """
    Create a generator for loading data
    """

    # shuffle the data
    perm = np.arange(len(x))
    np.random.shuffle(perm)
    x = x[perm]
    y = y[perm]

    numBatches = len(x) // batchSize

    x, y = torch.FloatTensor(x), torch.LongTensor(y)

    for i in range(numBatches):
        start = i * batchSize
        end = start + batchSize
        batchX = x[start: end]
        batchY = y[start: end]

        yield batchX, batchY

###########################################################################################
#### Start training
###########################################################################################

# You can use getData_naive to see the output of data
getData = getData_naive

# Decorator for computing time
def print_time(func):
    def decorated_func(*args, **kwargs):
        s = time.time()
        ret = func(*args, **kwargs)
        e = time.time()

        print(f"spend {e - s:.3f} s")
        return ret

    return decorated_func

@print_time
def train(train_data, epoch, model, loss_function, optimizer):
    print("\n" + "="*30 + "TRAIN" + "="*30)
    y_p = []
    y_t = []
    model.train()
    avg_loss = 0.0
    count = 0

    # need a batch dataloader
    for batch in getData(train_data[0], train_data[1], BATCH_SIZE):
        input_data = batch[0].to(DEVICE)
        target_data = batch[1].to(DEVICE)

        y_pred = model(input_data)
        print(y_pred.shape)
        print(target_data.shape)
        loss = loss_function(y_pred, target_data)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        #a better metric here
        predictions = y_pred.data.max(1)[1].cpu().tolist()
        y_p += predictions
        y_t += target_data.tolist()
        #a better metric here 
        count += 1
        if count % 100 == 0:
            print(f"epoch: {epoch}, iterations: {count}, loss: {loss.item():.3f}")

    print(f"the average loss after completion of {epoch} epochs is {avg_loss/count:.3f}")
    acc = genMetrics(y_t, y_p)
    print(f"acc: {acc:.3f}")
    print("="*60)

epochs = 4
for epoch in range(1, epochs+1):
    train([dummy_data, dummy_label], epoch, model, loss_function, optimizer)



==============================TRAIN==============================
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
the average loss after completion of 1 epochs is 2.309
acc: 0.101
spend 0.028 s

==============================TRAIN==============================
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torch.Size([512])
torch.Size([512, 10])
torc